## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-05-21-45-16 +0000,nypost,English nurse secretly married Manchester syna...,https://nypost.com/2025/10/05/world-news/engli...
1,2025-10-05-21-33-42 +0000,bbc,Conservatives to set out plan for £47bn of cut...,https://www.bbc.com/news/articles/c4gzv9j78dyo...
2,2025-10-05-21-33-27 +0000,nypost,NJ high school teacher charged with sexually a...,https://nypost.com/2025/10/05/us-news/nj-high-...
3,2025-10-05-21-32-22 +0000,nyt,Trump’s Gaza Peace Plan: What to Know,https://www.nytimes.com/2025/10/04/world/middl...
4,2025-10-05-21-32-19 +0000,nyt,"Drones, Democracy and the War in Ukraine",https://www.nytimes.com/2025/10/05/world/ukain...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2321/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
31,trump,34
32,gaza,11
128,national,10
129,guard,9
188,chicago,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
95,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...,124
71,2025-10-05-17-22-31 +0000,nypost,Kristi Noem slams ‘delusional’ Chicago mayor w...,https://nypost.com/2025/10/05/us-news/kristi-n...,85
19,2025-10-05-20-28-02 +0000,nypost,"Trump defies federal court, sends 300 Californ...",https://nypost.com/2025/10/05/us-news/trump-de...,84
13,2025-10-05-20-56-35 +0000,nyt,Trump Sends California National Guard Troops t...,https://www.nytimes.com/2025/10/05/us/trump-na...,72
46,2025-10-05-19-08-54 +0000,bbc,Trump authorises deployment of 300 National Gu...,https://www.bbc.com/news/articles/c2dnk0ee6pyo...,72


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
95,124,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...
176,41,2025-10-04-23-05-09 +0000,nypost,Youngkin calls for embattled Dem AG hopeful Ja...,https://nypost.com/2025/10/04/us-news/virginia...
175,33,2025-10-04-23-14-00 +0000,wsj,Defense Secretary Pete Hegseth’s emphasis on p...,https://www.wsj.com/politics/national-security...
11,32,2025-10-05-20-59-08 +0000,nypost,Jersey City Mayor Steve Fulop being eyed for b...,https://nypost.com/2025/10/05/us-news/jersey-c...
52,30,2025-10-05-18-46-55 +0000,nyt,"Strong-Armed by Trump, Netanyahu Embraces Gaza...",https://www.nytimes.com/2025/10/05/world/middl...
71,28,2025-10-05-17-22-31 +0000,nypost,Kristi Noem slams ‘delusional’ Chicago mayor w...,https://nypost.com/2025/10/05/us-news/kristi-n...
7,25,2025-10-05-21-24-00 +0000,wsj,The federal government shutdown is dragging to...,https://www.wsj.com/politics/policy/government...
2,24,2025-10-05-21-33-27 +0000,nypost,NJ high school teacher charged with sexually a...,https://nypost.com/2025/10/05/us-news/nj-high-...
130,24,2025-10-05-09-30-00 +0000,wsj,"As Laura Loomer, an influential loyalist to Pr...",https://www.wsj.com/politics/policy/laura-loom...
75,21,2025-10-05-16-44-45 +0000,bbc,Young Tories rate Kemi Badenoch's first year a...,https://www.bbc.com/news/videos/c2375x5endlo?a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
